-----

## Setup

In [1]:
use Math::SparseMatrix;
use Math::SparseMatrix::CSR;
use Math::SparseMatrix::Utilities;

use Data::Importers;
use Data::Summarizers;
use Data::Reshapers;
use Data::Generators;

use LLM::Configurations;

# In order to have "single origin of truth" in the comparison tables
use MONKEY-SEE-NO-EVAL;

------

## Random sparse matrices

Here we generate a random sparse matrix:

In [2]:
my $nrow = 5;
my $ncol = 8;
my $density = 0.35;
my $tol = 0.001;
my $type = 'CSR';

my $matrix1 = generate-random-sparse-matrix($nrow, $ncol, :$density, :$tol);

Math::SparseMatrix(:specified-elements(13), :dimensions((5, 8)), :density(0.325))

Here we display it (as text):

In [3]:
sink $matrix1.print()

–––––––––––––––––––––––––––––––––––––––––––––––––––
    0     1     2     3     4     5     6     7    
––┼––––––––––––––––––––––––––––––––––––––––––––––––
0 │ 0.059 .     .     0.747 .     .     .     .    
1 │ .     0.892 0.346 .     .     .     .     0.086
2 │ .     .     0.131 .     .     0.14  .     0.751
3 │ .     .     0.923 .     .     .     .     0.844
4 │ .     .     .     0.734 0.469 0.144 .     .    


Here is a "pretty print" in HTML:

In [4]:
#% html
$matrix1.to-html

,0,1,2,3,4,5,6,7
0,0.059,.,.,0.747,.,.,.,.
1,.,0.892,0.346,.,.,.,.,0.086
2,.,.,0.131,.,.,0.14,.,0.751
3,.,.,0.923,.,.,.,.,0.844
4,.,.,.,0.734,0.469,0.144,.,.


----

## Named rows and columns

In [5]:
my $matrix2 = Math::SparseMatrix.new(
    $matrix1, 
    row-names => random-pet-name($matrix1.nrow), 
    column-names => random-word($matrix1.ncol), 
    );

Math::SparseMatrix(:specified-elements(13), :dimensions((5, 8)), :density(0.325))

In [6]:
#% html
$matrix2.to-html

,dispensation,Anoectochilus,instalment,disinterment,conditioned,passport,rearguard,grotesque
Lucy,0.059,.,.,0.747,.,.,.,.
Chelsea,.,0.892,0.346,.,.,.,.,0.086
Anna Beans,.,.,0.131,.,.,0.14,.,0.751
Sevee,.,.,0.923,.,.,.,.,0.844
Dudley,.,.,.,0.734,0.469,0.144,.,.


Here is the result of a dot product with a dense vector: 

In [7]:
#% html
my $matrix3 = $matrix2.dot(rand xx $matrix2.columns-count);
$matrix3.to-html

,0
Lucy,0.5392729643957306
Chelsea,0.7096229249865935
Anna Beans,0.8369448680051328
Sevee,0.9016831963423682
Dudley,0.8271169924773762


Here is the result of the dot product with its transpose:

In [8]:
#% html
$matrix2.dot($matrix2.transpose).to-html

,Lucy,Chelsea,Anna Beans,Sevee,Dudley
Lucy,0.5614899999999999,.,.,.,0.548298
Chelsea,.,0.922776,0.10991199999999998,0.39194199999999996,.
Anna Beans,.,0.10991199999999998,0.600762,0.754757,0.02016
Sevee,.,0.39194199999999996,0.754757,1.564265,.
Dudley,0.548298,.,0.02016,.,0.779453


-----

## Test matrices

In [9]:
my $rmatCore = Math::SparseMatrix::CSR.new(rules => [[0, 0] => 1, [1, 1] => 2, [3, 2] => 3, [0, 3] => 4, [2, 4] => 2]);

my $rmat = Math::SparseMatrix.new(
  $rmatCore,  
  column-names => ["a", "b", "c", "d", "e"],
  row-names => ["A", "B", "C", "D"],
  dimension-names => ["U", "V"]);

Math::SparseMatrix(:specified-elements(5), :dimensions((4, 5)), :density(0.25))

In [10]:
#% html
$rmat.to-html(:!iv)

,a,b,c,d,e
A,1,.,.,4,.
B,.,2,.,.,.
C,.,.,.,.,2
D,.,.,3,.,.


----

## Dot product

### Matrix by vector

In [11]:
#% html
[ 
    $rmat.to-html(),
    $rmat[0].transpose.to-html,
    $rmat.dot($rmat[0].transpose).to-html
] ==> to-html-table

a b c d e A 1 . . 4 . B . 2 . . . C . . . . 2 D . . 3 . . A a 1 b . c . d 4 e . A A 17 B . C . D .

In [12]:
#% html
< 
    $rmat
    $rmat[0].transpose
    $rmat.dot($rmat[0].transpose)
>
==> { .map({ $_ => $_.&EVAL.to-html }) }() 
==> to-html-table

$rmat $rmat[0].transpose $rmat.dot($rmat[0].transpose) a b c d e A 1 . . 4 . B . 2 . . . C . . . . 2 D . . 3 . . A a 1 b . c . d 4 e . A A 17 B . C . D .

In [13]:
#% html
< 
    $rmat
    $rmat.core-matrix[0].transpose
    $rmat.dot($rmat.core-matrix[0].transpose)
>
==> { .map({ $_ => $_.&EVAL.to-html }) }() 
==> to-html-table

$rmat $rmat.core-matrix[0].transpose $rmat.dot($rmat.core-matrix[0].transpose) a b c d e A 1 . . 4 . B . 2 . . . C . . . . 2 D . . 3 . . 1 . . 4 . 0 A 17 B . C . D .

### Matrix by matrix

First we look into:
- Dot product to the _right_ of `Math::SparseMatrix` with a `Math::SparseMatrix::CSR` 
- Dot product to the _left_ of `Math::SparseMatrix` with a `Math::SparseMatrix::CSR`

In [14]:
#% html
<
    $rmat
    $rmat.core-matrix.transpose
    $rmat.dot($rmat.core-matrix.transpose)
    $rmat.transpose.dot($rmat.core-matrix)
>
==> { .map({ $_ => $_.&EVAL.to-html }) }() 
==> to-html-table

$rmat $rmat.core-matrix.transpose $rmat.dot($rmat.core-matrix.transpose) $rmat.transpose.dot($rmat.core-matrix) a b c d e A 1 . . 4 . B . 2 . . . C . . . . 2 D . . 3 . . 1 . . . . 2 . . . . . 3 4 . . . . . 2 . 0 1 2 3 A 17 . . . B . 4 . . C . . 4 . D . . . 9 0 1 2 3 4 a 1 . . 4 . b . 4 . . . c . . 9 . . d 4 . . 16 . e . . . . 4

This creates another `Math::SparseMatrix::CSR` object with _anonymous_ row and column names:

In [15]:
#my $dmat2 = random-real([0,4], $rmat.rows-count)».round xx $rmat.columns-count;
my $dmat2 = [2, 2, 4, 4, 4, 2, 3, 2, 0, 2, 1, 4, 4, 1, 1, 3, 1, 2, 1, 0].rotor(4);
my $rmat2 = Math::SparseMatrix.new(dense-matrix => $dmat2);

Math::SparseMatrix(:specified-elements(18), :dimensions((5, 4)), :density(0.9))

Next we look into two dot products of two `Math::SparseMatrix` objects:

In [16]:
#% html
<
    $rmat
    $rmat2
    $rmat.dot($rmat2)
    $rmat2.dot($rmat)
    $rmat.dot($rmat2).dot($rmat)
>
==> { .map({ $_ => $_.&EVAL.to-html }) }() 
==> to-html-table

$rmat $rmat2 $rmat.dot($rmat2) $rmat2.dot($rmat) $rmat.dot($rmat2).dot($rmat) a b c d e A 1 . . 4 . B . 2 . . . C . . . . 2 D . . 3 . . 0 1 2 3 0 2 2 4 4 1 4 2 3 2 2 . 2 1 4 3 4 1 1 3 4 1 2 1 . 0 1 2 3 A 18 6 8 16 B 8 4 6 4 C 2 4 2 . D . 6 3 12 a b c d e 0 2 4 12 8 8 1 4 4 6 16 6 2 . 4 12 . 2 3 4 2 9 16 2 4 1 4 . 4 2 a b c d e A 18 12 48 72 16 B 8 8 12 32 12 C 2 8 . 8 4 D . 12 36 . 6

### Arithmetic operations

The first three tables in this sub-section should be self-explanatory.

In [17]:
#% html
<
    $rmat
    $rmat.add(-2)
    $rmat.multiply(10)
    $rmat.multiply(10).add($rmat.multiply(2.33))
>
==> { .map({ $_ => $_.&EVAL.to-html }) }() 
==> to-html-table

$rmat $rmat.add(-2) $rmat.multiply(10) $rmat.multiply(10).add($rmat.multiply(2.33)) a b c d e A 1 . . 4 . B . 2 . . . C . . . . 2 D . . 3 . . a b c d e A -1 . . 2 . B . 0 . . . C . . . . 0 D . . 1 . . a b c d e A 10 . . 40 . B . 20 . . . C . . . . 20 D . . 30 . . a b c d e A 12.33 . . 49.32 . B . 24.66 . . . C . . . . 24.66 D . . 36.99 . .

In [18]:
#% html
my $rmat3 = Math::SparseMatrix.new($rmat2, row-names => $rmat.column-names, column-names => $rmat.row-names);

<
    $rmat
    $rmat3.transpose
    $rmat.add($rmat3.transpose)
    $rmat.multiply($rmat3.transpose)
>
==> { .map({ $_ => $_.&EVAL.to-html }) }() 
==> to-html-table

$rmat $rmat3.transpose $rmat.add($rmat3.transpose) $rmat.multiply($rmat3.transpose) a b c d e A 1 . . 4 . B . 2 . . . C . . . . 2 D . . 3 . . a b c d e A 2 4 . 4 1 B 2 2 2 1 2 C 4 3 1 1 1 D 4 2 4 3 . a b c d e A 3 4 . 8 1 B 2 4 2 1 2 C 4 3 1 1 3 D 4 2 7 3 . a b c d e A 2 . . 16 . B . 4 . . . C . . . . 2 D . . 12 . .

When an arithmetic operation can be performed on the underlying sparse arrays but the row names or column names do not coincide the names are dropped.

In [19]:
#% html
my $rmat4 = Math::SparseMatrix.new($rmat2, row-names => $rmat.column-names, column-names => $rmat.row-names.map('s.' ~ *));

<
    $rmat
    $rmat4.transpose
    $rmat.add($rmat4.transpose)
    $rmat.multiply($rmat4.transpose)
>
==> { .map({ $_ => $_.&EVAL.to-html }) }() 
==> to-html-table

$rmat $rmat4.transpose $rmat.add($rmat4.transpose) $rmat.multiply($rmat4.transpose) a b c d e A 1 . . 4 . B . 2 . . . C . . . . 2 D . . 3 . . a b c d e s.A 2 4 . 4 1 s.B 2 2 2 1 2 s.C 4 3 1 1 1 s.D 4 2 4 3 . 0 1 2 3 4 0 3 4 . 8 1 1 2 4 2 1 2 2 4 3 1 1 3 3 4 2 7 3 . 0 1 2 3 4 0 2 . . 16 . 1 . 4 . . . 2 . . . . 2 3 . . 12 . .

-----

## Row- and column binding

Row- and column binding are useful in various data analysis scenarios. 

When using row and column names there are couple of questions to be answered.

1. How duplication of row (column) names is handled?

2. How can we specify to ignore the row (column) names when the doing the binding?

Consider the following matrices:

In [20]:
#% html

my $rmat2 = Math::SparseMatrix.new($rmat, row-names => $rmat.row-names.map('s.' ~ *));
my $rmat3 = Math::SparseMatrix.new($rmat, column-names => $rmat.column-names.map('t.' ~ *));

<$rmat $rmat2 $rmat3>
==> { .map({ $_ => $_.&EVAL.to-html }) }() 
==> to-html-table


$rmat $rmat2 $rmat3 a b c d e A 1 . . 4 . B . 2 . . . C . . . . 2 D . . 3 . . a b c d e s.A 1 . . 4 . s.B . 2 . . . s.C . . . . 2 s.D . . 3 . . t.a t.b t.c t.d t.e A 1 . . 4 . B . 2 . . . C . . . . 2 D . . 3 . .

Here are row-binding of examples:

In [21]:
#% html

<$rmat.row-bind($rmat) $rmat.row-bind($rmat2)>
==> { .map({ $_ => $_.&EVAL.to-html }) }() 
==> to-html-table

$rmat.row-bind($rmat) $rmat.row-bind($rmat2) a b c d e A.1 1 . . 4 . B.1 . 2 . . . C.1 . . . . 2 D.1 . . 3 . . A.2 1 . . 4 . B.2 . 2 . . . C.2 . . . . 2 D.2 . . 3 . . a b c d e A 1 . . 4 . B . 2 . . . C . . . . 2 D . . 3 . . s.A 1 . . 4 . s.B . 2 . . . s.C . . . . 2 s.D . . 3 . .

In [22]:
#% html

<$rmat.column-bind($rmat) $rmat.column-bind($rmat3)>
==> { .map({ $_ => $_.&EVAL.to-html }) }() 
==> to-html-table

$rmat.column-bind($rmat) $rmat.column-bind($rmat3) a.1 b.1 c.1 d.1 e.1 a.2 b.2 c.2 d.2 e.2 A 1 . . 4 . 1 . . 4 . B . 2 . . . . 2 . . . C . . . . 2 . . . . 2 D . . 3 . . . . 3 . . a b c d e t.a t.b t.c t.d t.e A 1 . . 4 . 1 . . 4 . B . 2 . . . . 2 . . . C . . . . 2 . . . . 2 D . . 3 . . . . 3 . .

----

## Conversion to long form

The `Math::SparseMatrix` objects can be converted to "long form" using the method `tuples`:

In [23]:
say "Indexes : ", $rmat.tuples;
say "Names   : ", $rmat.tuples(:names);

Indexes : [(0 0 1) (0 3 4) (1 1 2) (2 4 2) (3 2 3)]
Names   : [(A a 1) (A d 4) (B b 2) (C e 2) (D c 3)]


Here is the "dataset" version:

In [24]:
#% html
<
    $rmat.tuples(:dataset)
    $rmat.tuples(:names,:dataset)    
>
==> { .map({ $_ => $_.&EVAL.&to-html(field-names => <i j x>) }) }() 
==> to-html-table

$rmat.tuples(:dataset) $rmat.tuples(:names,:dataset) i j x 0 0 1 0 3 4 1 1 2 2 4 2 3 2 3 i j x A a 1 A d 4 B b 2 C e 2 D c 3

**Remark:** By default both `:names` and `:dataset` are set to `False`.

Similar to the method `tuples` is the method `rules`:

In [25]:
say "Indexes : ", $rmat.rules;
say "Names   : ", $rmat.rules(:names);

Indexes : [(0 0) => 1 (0 3) => 4 (1 1) => 2 (2 4) => 2 (3 2) => 3]
Names   : [(A a) => 1 (A d) => 4 (B b) => 2 (C e) => 2 (D c) => 3]


**Remark:** Again, by default `:names` is set to `False`.